# Introduction
This is a hands-on guide to OpenLab's Python client...

[Web Client](https://live.openlab.app/)

[Web Client User Guide and Python Client](http://www.openlabdrilling.org/about-open-lab-drilling/webclient/#)

[OpenLab API](https://live.openlab.app/swagger/)



## Some Useful Keyboard Shortcuts
*(Help -> Keyboard Shortcuts for list of all shortcuts)*

`tab` Show all available methods for object

`Shift` `tab` Shows documentation for the selected method like argument info

`Up/Down` Navigate between cells

`Enter` Enter cell to edit

`Esc` Exit cell editing

`Ctrl` `Enter` Run selected cells code

`Ctrl` `Left/Right` Previous/Next word (Hold down `Shift` to select multiple words)

## Import libraries

In [ ]:
#pretty printing
import pprint as pr

#matlab style plotting
import matplotlib.pyplot as plt

#matlab style arrays
import numpy as np

# Network

## HTTP 
Hypertext Transfer Protocol, or HTTP, is the foundation application protocol for the internet (source - wikipedia). The 4 main methods that API's use, including Openlab, are:
    
    Post
    Put
    Get
    Delete

The OpenLab python client uses a library (oauth) to handle the dirty work and prepare the requests to the web client. Each call to openlab.http_client() creates a client instance from this library, and all HTTP methods can be called (like the following)

```
def whoami(self):

        """
        Returns information about the current user
        """
        r=self.client.get(self.url+"/users/whoami") 
        return self.standard_response(r, 200)
        
        ```
        

## Logging In

In [ ]:
import openlab

In [ ]:
# Copy and Paste the python generated script from live.openlab.app below
username=None
apikey=None
licenseguid=None

In [ ]:
session = openlab.http_client(username=username, apikey=apikey,licenseguid=licenseguid) #this intializes an http client that can be used for an entire openlab session

## JSON Data
JavaScript Object Notation, or JSON, is a text data structure which is both lightweight and easible parsable for computers, but also easy for humans to read and write (source - json.org). Which is why it is so popular. It is defined as key/value pairs seperated by a colon 

In [ ]:
# Information about userprint(session.whoami())
session.whoami()

In [ ]:
# Return type of session.whoami() is a dictionary
type(session.whoami())

In [ ]:
# We can see all the available keys using .keys() method
session.whoami().keys()

In [ ]:
# accessing a value associated to a dictionary
session.whoami()['Name']

# Configurations

In [ ]:
configurations = session.configurations() # returns a list of all configurations
msg = "Configuration name is: {}"
for config in configurations: #lists can be accessed by indices or with for loops like here
    pr.pprint(msg.format(config['Name']))

In [ ]:
print(session.configuration_id("test")) # note that you must put in configiuration names that you have created in the client
print(session.configuration_id("test2")) # for me, i had two configurations titled "test" and "test2"

In [ ]:
# or we could acces the value associated with the 'Name' key for each config without knowing/inputting anything manually
for config in configurations:
    config_id = session.configuration_id(config['Name'])
    print(config_id)

In [ ]:
# example of getting information that is nested 
for config in configurations: #configurations is a list with each index (i.e. config) being a dictionary
    print(config['Name']) 
    simulations = session.configuration_simulations(config['ConfigurationID'])
    for simulation in simulations:
        print("\t" + simulation['Name'])

# Simulation

### Initializing Simulation

In [ ]:
sim_name = "Simulation from Notebook Tutorial"
config_name = "test"
initial_bit_depth = 2500
influx_mode = openlab.default_geopressure_gradient #optional positional argument that you can pass in. Otherwise, no influx mode is selected
sim = session.create_simulation(config_name, sim_name, initial_bit_depth)
sim.end_simulation_on_exiting = False # Don't end simulation upon code exit. Default is true

### Stepping/running simulation

In [ ]:
#step 100 time steps without setting anything and see that it ran/is running in web client
for i in range(1,100):
    sim.step(i)

### Setting setpoints
***Important!*** All units must be in SI

##### Available setpoints

In [ ]:
print(openlab.setpoints.all_setpoints())

In [ ]:
sim.setpoints.SurfaceRPM = 2 #HZ
sim.setpoints.TopOfStringVelocity = 0.02 #m/s
sim.setpoints.DesiredROP = 0.02 #m/s
sim.setpoints.FlowRateIn = 2500/60000 #converting l/min to m^3/s

In [ ]:
start = sim.current_step()+1
for i in range(start,start + 100):
    sim.step(i)

### Getting Results

#### Available Results
***Important!*** All units are returned in SI

In [ ]:
print(openlab.results.all_results())

In [ ]:
tags = ["SPP", "DownholePressure","SurfaceTorque", "BitDepth", "WOB","DrillstringTension","DrillstringBucklingLimit"]

In [ ]:
start = sim.current_step()+1
for i in range(start,start+50):
    sim.step(i)
    sim.get_results(i, tags) # need to do this do avoid client waisting resources getting results user doesn't want

In [ ]:
sim.results.SPP

In [ ]:
## seeing those results
msg = "The last torque value was {}"
print(msg.format(sim.results.SurfaceTorque[i])) ## print the last torque

In [ ]:
#or print a range of torques truncated with no decimals
msg = "Torque at time {} was {:0.0f}"
for timestep in range(i-10, i+1): # range doesn't include the endpooint so we need to add 1
    print(msg.format(timestep, sim.results.SurfaceTorque[timestep])) #{0.0f} is a formatter meaning 0 decimal points

### Non Depth Based Results

In [ ]:
# Get simulation results for time range (or for a previously run simulation)
from_time = 95
to_time = 105
tags = ["SPP", "FlowRateIn"]
results = session.get_simulation_results(sim.sim_id,from_time,to_time,True, tags) # see cell below for explanation of the True
pr.pprint(results)

### Depth Based Results

In [ ]:
# Setting true in get_simulation_results will exlude depth based values for all but last setpoint.
from_time = 95
to_time = 105
tags = ["DrillstringTension", "SPP", "FlowRateIn"]
depth_results = session.get_simulation_results(
    sim.sim_id , from_time , to_time , True , tags) # Pass in False to get all depth based values
pr.pprint(depth_results) #notice when True, only the last requested setpoint has depth based-values

# Plotting

### Plot Non-depth Results 
***(using list comprehension)***

List comprehension provides a more concise way to make a new list from e.g. a dictionary of dictionaries
Normally, this is done with a for loop like:

`
new_list = list()
for i in old_list.keys():
    new_list.append(old_list[i][key])
    `
    
which can be replaced with a one liner (which for me is harder to read)

`
new_list = [old_list[i][key] for i in old_list.keys()]`

In [ ]:
#get results for the two plots we want (depth and non-depth based results)
startTime = 95
endTime = 120
tags = ["SPP", "FlowRateIn", "DrillstringTension", "DrillstringBucklingLimit"]
results = session.get_simulation_results(sim.sim_id,startTime,endTime,True, tags)

#create a list of timesteps for the x-axis
x = (list(results.keys()))

#list comprehensions for the y-axes
flow = [results[t]['FlowRateIn'] for t in x]
spp = [results[t]['SPP'] for t in x]

#initialize figure
fig = plt.figure()
fig.legend(['Flow', 'SPP'], loc = 'upper center')

#create first axis
ax1 = fig.add_subplot(111)
ax1.set_ylabel('Flow')
ax1.plot(x, flow, 'k')

#initialize a second 'twin' axis that shares the x-axis
ax2 = ax1.twinx()
ax2.set_ylabel('SPP')
ax2.plot(x, spp, 'r-')

### Plot depth based results 
(***using lambda function***)

In [ ]:
# map a lamda function
extract_depth = lambda data: (data['v'], data['d']) #'lambda' signals next thing(s) are argument(s) and it returns what is after ':'
tensions = map(extract_depth, results[120]["DrillstringTension"])
limits = map(extract_depth, results[120]["DrillstringBucklingLimit"])

plt.gca().invert_yaxis() #invert y-axis
plt.plot(*zip(*tensions),'k',*zip(*limits),'r--')

# Ending Simulation
For When `Simulation.end_simulation_on_exiting = False` *(True by default)*

In [ ]:
sim.stop()